In [6]:
#import packages
# from urllib import request
# from bs4 import SoupStrainer, BeautifulSoup
import bs4 as bs
import requests

import re
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import datetime as dt
import os

if not os.path.exists('./results'):
    os.makedirs('./results')

def add_to_json(df, name_str):
        filename = './results/'+name_str+'.json'
#         print(filename)
        if not os.path.exists(filename):
            df.to_json(filename, orient="records")
        else:
            df_ = pd.concat([pd.read_json(filename), df])
            df_.to_json(filename, orient="records")
            
from selenium.common.exceptions import TimeoutException

In [2]:
# # class MyOpener(FancyURLopener):
# #     version = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.152 Safari/537.36'
# # openurl = MyOpener().open

# def data_from_page_senadores(url):
#     """
#     Function to scrap relevant data from election results webpage.
    
#     Input url and outputs a dataframe row with a bunch of information from the page.
#     """
    
#     url = openurl(url)
#     page = BeautifulSoup(url.read())

In [3]:
# Formato de URLs

# url = 'https://resultados.gob.ar/elecciones/3/0/1/-1/-1' # DIP NACS
# url = 'https://resultados.gob.ar/elecciones/2/0/1/-1/-1' # SEN NACS
# https://resultados.gob.ar/elecciones/3/128588/1/-1/-1/
### Ultima mesa

## URL de un DPTO
# url = 'https://resultados.gob.ar/elecciones/3/300/1/-1/-1/'


In [4]:
# Tiempo medio de scrapeo
# np.mean(np.array([4.48,2.6,2.18,1.91,2.64,2.04,2.18,1.98,2.04,1.93,2.2,1.88,2.65,2.1,2.36,2.94,1.96]))
# 2.36*128000/3600/24

# 3 dias y medio scrapeando...
# 3.4962962962962965

In [5]:

# for ix in [10000, 10001]:
# for ix in range(25, 27):
# for ix in [str(int(i)) for i in np.round(10**np.arange(1, 5.25, .025))]:
# for ix in [str(int(i)) for i in range(25, 6555)]:
for ix in [str(int(i)) for i in range(584, 6555)]:
    if int(ix) 10 == 0: print(ix)

    url = 'https://resultados.gob.ar/elecciones/3/'+str(ix)+'/1/-1/-1/'

    driver = webdriver.Firefox()
#     driver.implicitly_wait(3)
    driver.get(url)

#     time.sleep(3)
    ##############################

    # <div class="MigasYAct__Container-dnAgYZ dtcSOp containerMigas">
    # <ol class="breadcrumbs-list">
    try: 
        start = time.time()
        wait = WebDriverWait(driver, 12)
        element_found = wait.until(lambda x: x.find_element_by_class_name("MigasYAct__Container-dnAgYZ").is_displayed())
    #     TimeoutException
        end = time.time(); if int(ix) 10 == 0: print(round(end - start, 2))

        region_container = driver.find_element_by_class_name("MigasYAct__Container-dnAgYZ")
        region_text = region_container.find_element_by_class_name("breadcrumbs-list").text

        df = pd.DataFrame(region_text.split('\n>\n')).T; df.columns = ['nivel_' + str(col) for col in df.columns]
        df['ix'] = ix
        df['timestamp'] = dt.datetime.today()

    #     display(df)
        add_to_json(df, 'regiones')

        ##############################

        # <div class="Results__InfoPorAgrupacion-bmdcyG hYVsOk pagina" name="agrupaciones">
        results = driver.find_element_by_class_name("Results__InfoPorAgrupacion-bmdcyG")
        # # <div class="InfoAgrupacion__Contenido-huLckG bByeTM">
        # info_agrupaciones = results.find_elements_by_class_name("InfoAgrupacion__Container-dqzjKW")

        # <div class="InfoAgrupacion__Container-dqzjKW dlomjH">
        info_agrupaciones = results.find_elements_by_class_name("InfoAgrupacion__Contenido-huLckG")

        values = []
        campos_str = ['titleNombreTerr', 'porcAgr', 'numAgr',
                     'nombreTerr', 'porcListas', 'numListas']

        for i in info_agrupaciones:

            for campo in campos_str:
                try:
                    globals()[campo] = i.find_elements_by_class_name(campo)[0].text
                except IndexError:
                    globals()[campo] = np.nan

            values += [[titleNombreTerr, porcAgr, numAgr,
                         nombreTerr, porcListas, numListas]]

        resultados = pd.DataFrame(values, columns = campos_str)
        resultados['ix'] = ix
        add_to_json(resultados, 'resultados')

        ##############################


        ## Opcion 2
        censo_str = ['Electores totales', 'Votantes totales', 'Mesas escrutadas', 'Mesas totales', 
                    'Votos válidos', 'Afirmativos', 'En blanco', 'Votos nulos', 'Votos recurridos, impugnados y comando']
        censo_vals = []

        for i in driver.find_elements_by_class_name("content.contentCenso"):
            censo_vals += [i.text]

        censo = pd.DataFrame(censo_vals, index = censo_str).T
        censo['ix'] = ix
        add_to_json(censo, 'censo')
    
    except TimeoutException:
        print('TimeoutException for ix: '+ str(ix))
    
    driver.close()
    

25
3.7
26
2.89
27
3.28
28
3.04
29
3.13
30
3.24
31
3.59
32
2.97
33
2.93
34
2.86
35
3.38
36
2.87
37
2.51
38
2.49
39
2.56
40
2.54
41
2.56
42
2.78
43
2.99
44
2.5
45
2.77
46
2.8
47
3.07
48
3.17
49
2.8
50
2.64
51
2.94
52
2.73
53
3.2
54
3.33
55
3.24
56
3.09
57
2.9
58
3.2
59
3.47
60
2.82
61
2.93
62
2.92
63
2.77
64
2.39
65
2.77
66
2.21
67
2.94
68
2.42
69
2.77
70
2.41
71
2.62
72
3.12
73
2.55
74
2.9
75
3.18
76
3.13
77
2.66
78
5.01
79
2.91
80
3.25
81
3.22
82
3.16
83
2.91
84
3.04
85
3.21
86
2.94
87
3.17
88
3.01
89
2.99
90
3.14
91
2.79
92
3.03
93
3.2
94
3.03
95
3.1
96
2.69
97
3.1
98
2.99
99
3.36
100
3.05
101
2.87
102
2.96
103
2.54
104
2.99
105
2.93
106
3.26
107
2.35
108
2.81
109
3.21
110
2.85
111
2.74
112
2.71
113
2.96
114
2.71
115
2.63
116
2.86
117
2.87
118
2.94
119
2.61
120
2.78
121
3.05
122
2.86
123
4.19
124
2.83
125
2.92
126
3.32
127
2.95
128
2.63
129
2.81
130
2.78
131
2.87
132
2.69
133
2.76
134
3.09
135
2.73
136
3.03
137
2.82
138
2.58
139
2.78
140
2.74
141
2.46
142
2.97
143
2.65
144
2.8
145
2.7

NameError: name 'TimeoutException' is not defined

In [11]:
display(pd.read_json('./results/regiones.json'))
display(pd.read_json('./results/resultados.json'))
display(pd.read_json('./results/censo.json'))

,nivel_0,nivel_1,nivel_2,ix,timestamp,nivel_3,nivel_4
0,Argentina,Buenos Aires,Sección Primera,25,2021-09-21 02:05:33.308,None,None
1,Argentina,Buenos Aires,Sección Segunda,26,2021-09-21 02:05:54.713,None,None
2,Argentina,Buenos Aires,Sección Tercera,27,2021-09-21 02:06:16.407,None,None
3,Argentina,Buenos Aires,Sección Cuarta,28,2021-09-21 02:06:36.674,None,None
4,Argentina,Buenos Aires,Sección Quinta,29,2021-09-21 02:06:57.238,None,None
...,...,...,...,...,...,...,...
554,Argentina,Buenos Aires,Sección Segunda,579,2021-09-21 04:36:34.607,Arrecifes,Arrecifes
555,Argentina,Buenos Aires,Sección Tercera,580,2021-09-21 04:36:49.174,Avellaneda,Avellaneda
556,Argentina,Buenos Aires,Sección Quinta,581,2021-09-21 04:37:05.275,Ayacucho,Ayacucho
557,Argentina,Buenos Aires,Sección Séptima,582,2021-09-21 04:37:19.735,Azul,Azul


,titleNombreTerr,porcAgr,numAgr,nombreTerr,porcListas,numListas,ix
0,JUNTOS,"37,79%",1.098.696,JUNTOS,None,None,25
1,None,None,None,ES JUNTOS,"67,39%",740.471 votos,25
2,None,None,None,DAR EL PASO,"32,60%",358.225 votos,25
3,FRENTE DE TODOS,"33,18%",964.773,FRENTE DE TODOS,None,None,25
4,None,None,None,CELESTE Y BLANCA,100%,964.773 votos,25
...,...,...,...,...,...,...,...
15754,None,None,None,JUSTICIA Y DIGNIDAD,100%,162 votos,583
15755,LABORISTA,"0,01%",18,LABORISTA,None,None,583
15756,None,None,None,BLANCA,100%,18 votos,583
15757,MORAL Y PROGRESO,"0,01%",18,MORAL Y PROGRESO,None,None,583


,Electores totales,Votantes totales,Mesas escrutadas,Mesas totales,Votos válidos,Afirmativos,En blanco,Votos nulos,"Votos recurridos, impugnados y comando",ix
0,4.468.873,2.955.865,12.462,12.819,2.906.839,2.791.079,115.76,45.053,3.973,25
1,618.880,414.672,1.77,1.829,407.895,385.267,22.628,6.334,443,26
2,4.435.592,2.948.561,12.262,12.727,2.904.436,2.784.972,119.464,40.192,3.933,27
3,524.739,356.883,1.561,1.594,351.100,329.136,21.964,5.462,321,28
4,1.231.163,781.650,3.51,3.616,764.598,720.484,44.114,15.72,1.332,29
...,...,...,...,...,...,...,...,...,...,...
554,25.107,17.730,74,75,17.434,16.655,779,271,25,579
555,287.833,195.261,800,826,192.765,186.765,6.000,2.217,279,580
556,17.977,11.369,60,60,11.108,10.085,1.023,245,16,581
557,57.859,36.950,169,174,35.912,32.402,3.510,989,49,582


In [ ]:
xx

In [ ]:
## Opcion 1

# # <div class="InfoDatosGenerales__Container-ituRjI gwsoDF sCol4-md sCol12M-sm datosGeneralesContainer infoGeneral">
# datosGenerales = driver.find_element_by_class_name("InfoDatosGenerales__Container-ituRjI")
# print(datosGenerales.text.split('\n'))

# # <div class="InfoVotacion__ParteSup-fctFJJ hWYMGK">
# infoBlancos = driver.find_element_by_class_name("InfoVotacion__ParteSup-fctFJJ")
# print(infoBlancos.text.split('\n'))

# # <div class="InfoVotacion__ParteMedia-fA-dpov kcjBXI">
# infoNulos = driver.find_element_by_class_name("InfoVotacion__ParteMedia-fA-dpov")
# print(infoNulos.text.split('\n'))

# # <section class="Votos__Container-cFJeGu eeaVWG recImpCom">
# infoRecurridos = driver.find_element_by_class_name("Votos__Container-cFJeGu")
# print(infoRecurridos.text.split('\n'))


In [ ]:
driver.close()

In [ ]:
xx

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title
# elem = driver.find_element_by_name("q")
elem = driver.find_element_by_tag_name("div")
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source

driver.close()

In [ ]:
# given an element defined as:

# <input type="text" name="passwd" id="passwd-id" />
# <menu-7 id="menu-7" class="Dropdownbox__List-dDVmyU ggnRIT" role="listbox" aria-labelledby="downshift-83-label">

# you could find it using any of:

# element = driver.find_element_by_id("menu-7")
# element = driver.find_element_by_xpath("//menu-7[@id='menu-7']")
# element = driver.find_element_by_css_selector("menu-7#menu-7")


# <div id="root">
# element = driver.find_element_by_id("root")
# <nav class="NavMenu__Container-dZELt…_SNavMenu-cmglKK gpvJyq">
# <svg class="NavMenu___StyledFiltroIcon-bfIXDX iuaeqU" width="44" height="44" viewBox="0 0 44 44" fill="none" xmlns="http://www.w3.org/2000/svg">


url = 'https://resultados.gob.ar/elecciones/3/3137/1/-1/-1/Ciudad-Aut%C3%B3noma-de-Buenos-Aires/Comuna-12/00125#agrupaciones'

driver = webdriver.Firefox()
driver.get(url)

# element = driver.find_element_by_id("root")
# elem2 = element.find_element_by_tag_name("nav")
# elem2.click()

# elem1 = driver.find_element_by_id("menu-7")
# elem2 = driver.find_element_by_xpath("//menu-7[@id='menu-7']")
# elem3 = driver.find_element_by_css_selector("menu-7#menu-7")


# element.click()
# driver.close()


In [ ]:
url = 'https://resultados.gob.ar/elecciones/3/0/1/-1/-1' # DIP NACS
# url = 'https://resultados.gob.ar/elecciones/2/0/1/-1/-1' # SEN NACS
# https://resultados.gob.ar/elecciones/3/128588/1/-1/-1/
### Ultima mesa

## Lista de provincias


driver = webdriver.Firefox()
driver.get(url)

import time
time.sleep(3)


element = driver.find_element_by_id("root")
elem2 = element.find_element_by_class_name("NavMenu__Container-dZELtJ")
elem2.click()

# <button class="Dropdownbox__Opener-tLZCp byDCKs" aria-expanded="false" aria-controls="menu-2" aria-label="Escoja una provincia">
elem3 = elem2.find_element_by_class_name("Dropdownbox__Opener-tLZCp")
elem3.click()

elem4 = elem2.find_element_by_id("menu-2")
print(elem4.text)


In [ ]:
driver.close()

In [ ]:
elem2.find_element_by_id("menu-2")

In [ ]:
url = 'https://resultados.gob.ar/elecciones/3/3137/1/-1/-1/Ciudad-Aut%C3%B3noma-de-Buenos-Aires/Comuna-12/00125#agrupaciones'

driver = webdriver.Firefox()
driver.get(url)

import time
time.sleep(3)

# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.tag, "nav"))
#     )
# except:
#     pass

element = driver.find_element_by_id("root")

# elem2 = element.find_element_by_tag_name("nav")

# <nav class="NavMenu__Container-dZELtJ eBAoEc Header__SNavMenu-cmglKK gpvJyq">
elem2 = element.find_element_by_class_name("NavMenu__Container-dZELtJ")

elem2.click()

elem2 = element.find_element_by_class_name("NavMenu__Container-dZELtJ")

# <button id="downshift-4-toggle-button" tabindex="-1" style="background: rgba(0, 0, 0…ne repeat scroll 0% 0%;" aria-hidden="true">
# elem3 = elem2.find_element_by_id("downshift-4-toggle-button")
# elem3.click()

# <input id="downshift-4-input" class="Dropdownbox__SInput-jSpFAX csVrte" 
# aria-autocomplete="list" aria-controls="downshift-4-menu" aria-labelledby="downshift-4-label" 
# autocomplete="off" placeholder="Introduzca un local de comicio" value="">

# <ul id="menu-7" class="Dropdownbox__List-dDVmyU ggnRIT" role="listbox" aria-labelledby="downshift-4-label">
elem4 = elem2.find_element_by_id("menu-7")
elem4.text
driver.close()

In [ ]:
url = 'https://resultados.gob.ar/elecciones/3/3137/1/-1/-1/Ciudad-Aut%C3%B3noma-de-Buenos-Aires/Comuna-12/00125#agrupaciones'

driver = webdriver.Firefox()
driver.get(url)

import time
time.sleep(3)

# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.tag, "nav"))
#     )
# except:
#     pass

element = driver.find_element_by_id("root")

# elem2 = element.find_element_by_tag_name("nav")

# <nav class="NavMenu__Container-dZELtJ eBAoEc Header__SNavMenu-cmglKK gpvJyq">
elem2 = element.find_element_by_class_name("NavMenu__Container-dZELtJ")

elem2.click()

elem2 = element.find_element_by_class_name("NavMenu__Container-dZELtJ")

# <button id="downshift-4-toggle-button" tabindex="-1" style="background: rgba(0, 0, 0…ne repeat scroll 0% 0%;" aria-hidden="true">
# elem3 = elem2.find_element_by_id("downshift-4-toggle-button")
# elem3.click()

# <input id="downshift-4-input" class="Dropdownbox__SInput-jSpFAX csVrte" 
# aria-autocomplete="list" aria-controls="downshift-4-menu" aria-labelledby="downshift-4-label" 
# autocomplete="off" placeholder="Introduzca un local de comicio" value="">

# <ul id="menu-7" class="Dropdownbox__List-dDVmyU ggnRIT" role="listbox" aria-labelledby="downshift-4-label">
elem4 = elem2.find_element_by_id("menu-7")
elem4.text

In [ ]:
driver.close()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Firefox()
driver.get(url)
# assert "Resultados" in driver.title

# elem = driver.find_element_by_tag_name("div")
elem = driver.find_element_by_id("root")
# elem.find_element_by_id("root")

# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
driver.close()

In [ ]:
elem?

In [ ]:
assert "Resultados" in driver.title
